In [11]:
## imports

import numpy as np
import numpy.ma as ma
import xarray as xr

import sys
sys.path.insert(0,"/Users/auraoupa/Work/git/lupa/vorticity-variance")
import GriddedData

import matplotlib.pyplot as plt

# palette
import matplotlib.cm as mplcm

seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

import glob

In [12]:
## Dataset

dir="/Users/auraoupa/Data/GLORYS2V4/2017/"



In [13]:
def compute_average_fine_scale_variance(files):
    ''' from a list of files containing one month of data compute variance and average in 1°x1°x1 month boxes
    '''
    hpcurl = xr.open_mfdataset(files,concat_dim='time_counter',decode_times=False)['socurloverf_filt']
    navlat = xr.open_mfdataset(files,concat_dim='time_counter',decode_times=False)['nav_lat'][0]
    navlon = xr.open_mfdataset(files,concat_dim='time_counter',decode_times=False)['nav_lon'][0]
    hpcurl2 = hpcurl ** 2
    hpcurl2m = hpcurl2.mean(axis=0,keep_attrs=True)
    navlat2=np.array(navlat).squeeze()
    navlon2=np.array(navlon).squeeze()
    mgrd = GriddedData.grid2D(navlat=navlat2, navlon=navlon2)
    crs = GriddedData.grdCoarsener(mgrd,crs_factor=4)
    hpcurl2mc = crs.return_ravel(np.asarray(hpcurl2m))
    hpcurl2mcm = np.mean(hpcurl2mc,axis=-3)
    latcrs=crs.return_ravel(np.asarray(navlat2))
    loncrs=crs.return_ravel(np.asarray(navlon2))
    latcrsm=np.mean(latcrs,axis=-3)
    loncrsm=np.mean(loncrs,axis=-3)
    return loncrsm,latcrsm,hpcurl2mcm,hpcurl2m



In [14]:
def compute_and_save(month):
    mm=str(month).zfill(2)
    filt_files_month=mm+"/GLORYS2V4NATL_1dAV_2017"+mm+"*_curloverf_filt-n4-f0.25.nc"
    tf_files_month=dir+filt_files_month
    loncrsm, latcrsm, boxvarcurl, hpvarm=compute_average_fine_scale_variance(tf_files_month)
    fine_scale_variance = xr.DataArray(boxvarcurl,dims=['xcrsm','ycrsm'],name="vort_var_boxcar")
    lat_crsm = xr.DataArray(latcrsm,dims=['xcrsm','ycrsm'],name="latcrsm")
    lon_crsm = xr.DataArray(loncrsm,dims=['xcrsm','ycrsm'],name="loncrsm")
    fine_scale_variance.to_dataset().to_netcdf(path=dir+'/fine_scale_vorticity_variance_boxaverage_natl_y2017m'+str(mm)+'.nc',mode='w',engine='scipy')
    lat_crsm.to_dataset().to_netcdf(path=dir+'/fine_scale_vorticity_variance_boxaverage_natl_y2017m'+str(mm)+'.nc',mode='a',engine='scipy')
    lon_crsm.to_dataset().to_netcdf(path=dir+'/fine_scale_vorticity_variance_boxaverage_natl_y2017m'+str(mm)+'.nc',mode='a',engine='scipy')



In [16]:
compute_and_save(1)

In [19]:
for m in np.arange(1,13):
    print m
    compute_and_save(m)

1
2
3
4
5
6
7
8
9
10
11
12
